In [1]:
from constants import REFINED_MITRE_PROMPT_TEMPLATE
result = REFINED_MITRE_PROMPT_TEMPLATE.format(
    advisory_content="Sample advisory content for testing",
    techniques_to_analyze="sample techniques to analyze")
print(result)


You are a cybersecurity expert analyzing an ICS (Industrial Control Systems) security advisory.
Your task is to map this advisory to the most relevant MITRE ATT&CK techniques.

IMPORTANT: You have been provided with pre-filtered candidate techniques that are most likely relevant.
Focus your analysis on these candidates, but you may also suggest if none are truly appropriate.

Advisory Content:
Sample advisory content for testing

Candidate MITRE ATT&CK Techniques (pre-filtered for relevance):
sample techniques to analyze

Instructions:
1. Analyze the advisory for attack vectors, vulnerabilities, and potential impacts
2. Map to 1-3 most relevant techniques from the candidates provided

Only respond with a valid JSON object in this exact format and nothig else
{
  "type": "object",
  "properties": {
    "mapped_techniques": {
      "type": "array",
      "description": "List of MITRE ATT&CK technique IDs that apply the MOST to the advisory. This could be zero, one or more techniques but